In [14]:
cd ../src

/gpfs/home/gdb20/code/batch-manager/src


In [15]:
import input_files
import input_combi
import helpers
import os

In [21]:
global_configs = { '' : {
        'num_cores' : 8,
        'mem_per_cpu_GB' : 4,
        'runtime' : '2-00:00:00',
        #'!max_jobs' : 10, #this doesn't do anything yet
        #'solvent' : 'thf', #this is what Mike told me we use
    }
}

thf = { 'thf' : {
    'solvent' : 'thf',
    }
}

xyz_root = '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules'
mike_dir = os.path.join(xyz_root,'')

mike_mols = input_combi.xyz_files_from_directory(mike_dir)
mike_mols['!directories'] = True



orbital_and_uno_occs_blocks = {
    'output' : [
        'print[P_UNO_OccNum] 1'
    ],    
    'plots'  : [
        'dim1 100',
        'dim2 100',
        'dim3 100',
        'Format Gaussian_Cube',
        'SpinDens("spindensity");',
        'ElDens("electrondensity");',
    ]
}   

tddft_block = {
    'tddft' : [
        'maxdim 5',
        'nroots 50',
        'DoNTO true',
        'NTOStates 1,2,3,4,5,6,7,8,9,10', #somewhat arbitrary choice
    ],
}

diffuse_linear_dependency_blocks = {
    'scf' : [
        'sthresh 1e-6',
    ],
    'basis' : [
        'PCDTrimBas overlap',
        'PCDTrimAuxJ coulomb',
        'PCDTrimAuxJK coulomb',
        #'PCDTrimAuxC coulomb', #for use with correlated calculations
        'PCDThresh -1', #-1 means automatically select threshold
    ]
}


orca_defaults = {
    'program' : 'ORCA',
    'integration_grid' : 'DEFGRID2', #default settings
    'scf_tolerance' : 'TightSCF', #see if they work in ORCA 6
}

orca_singlepoint = orca_defaults.copy()
orca_singlepoint.update({      
    'natural_orbitals' : True,
    'runtime' : '0-05:00:00',
    'blocks' : orbital_and_uno_occs_blocks,   
    'run_type' : None,
})

orca_opt_freq = orca_defaults.copy()
orca_opt_freq.update({
    'runtime' : '3-00:00:00',
    'run_type' : 'OPT FREQ',
})

orca_tddft = orca_defaults.copy()
orca_tddft.update({
    'runtime' : '4-00:00:00',
    'run_type' : None,
    'blocks' : tddft_block,
})


orca_nics = orca_defaults.copy()
orca_nics.update({
    'runtime' : '3-00:00:00', #some of these will fail, dang. took like, AN HOUR last time we did this, so weird
    'run_type' : 'NMR', #GIAO is the default for ORCA
})

gaussian_nics = {
    'program'  : 'Gaussian',
    'runtime'  : '3-00:00:00',
    'run_type' : 'NMR=GIAO', 
}

gaussian_nics_preprocessing = {
    'num_cores' : 1,
    'mem_per_cpu_GB' : 3,
    'runtime' : '0-00:05:00',
    'program'  : 'pyAroma',
    'cc_program' : 'Gaussian',
}

orca_nics_preprocessing = {
    'num_cores' : 1,
    'mem_per_cpu_GB' : 3,
    'runtime' : '0-00:05:00',
    'program'  : 'pyAroma',
    'cc_program' : 'ORCA',
}


r2scan_3c = {
    'functional' : 'r2SCAN-3c',    
    'basis' : None,
    'aux_basis' : None,
}

cam_b3lyp = {
    'functional' : 'CAM-B3LYP',
    'basis' : '6-31G(d,p)',
    'aux_basis' : 'autoaux',
}

wb97x_3c = {
    'functional' : 'wB97X-3c',
    'basis_set' : None,
    'aux_basis' : None,
}
neutral_singlet = {
    'charge' : 0,
    'spin_multiplicity' : 1,
    'broken_symmetry' : False,
}

neutral_triplet = {
    'charge' : 0,
    'spin_multiplicity' : 3,
    'broken_symmetry' : False,
}

neutral_singlet_brokensym = {
    'charge' : 0,
    'spin_multiplicity' : 3,
    'broken_symmetry' : True,
}

neutral_singlet_guessmix = {
    'charge' : 0,
    'spin_multiplicty' : 1,
    'uks' : True,
    'keywords' : ['guess=mix']
}

crest_quick_settings = {
    'program' : 'CREST',
    'functional' : 'gfn2',
    'quick' : 'quick',
    'reopt' : True,
    'cluster' : True,
    'noreftopo' : True, #might regret this
}

xtb_cm_states = {
    '0_1' : neutral_singlet,
    '0_3': neutral_triplet,
}
orca_cm_states = xtb_cm_states.copy()
orca_cm_states.update({
    '0_1' : neutral_singlet_brokensym,
})

gaussian_cm_states = xtb_cm_states.copy()
gaussian_cm_states.update({
    '0_1' : neutral_singlet_guessmix,
})


In [22]:
crest_gfn2__gfnff_squick = crest_quick_settings.copy()
crest_gfn2__gfnff_squick.update({        
    'functional' : 'gfn2//gfnff',
    'quick' : 'squick',
})

crude_confsearch_optfreq = {
    'crest_gfn2__gfnff_squick' : crest_gfn2__gfnff_squick,
    'xtb_gfn2_opt_freq' : {
        'program' : 'XTB',
        'functional' : 'gfn2',
        'run_type' : 'ohess', #opt-freq
        '!coords_from' : '../crest_gfn2__gfnff_squick',
        '!xyz_file' : 'crest_best.xyz',
    },
}

#SINGLEPOINTS
r2scan_3c_triplet_sp = orca_singlepoint.copy()
r2scan_3c_triplet_sp.update({          
        **r2scan_3c,
        **neutral_triplet,   
        '!coords_from' : '../xtb_gfn2_opt_freq',
        '!xyz_file' : 'xtbopt.xyz',
        #'!overwrite' : True, #VERY DANGEROUS, comment out when not using     
})

r2scan_3c_singlet_sp = orca_singlepoint.copy()
r2scan_3c_singlet_sp.update({ 
        **r2scan_3c,
        **neutral_singlet_brokensym,
        '!coords_from' : '../xtb_gfn2_opt_freq',
        '!xyz_file' : 'xtbopt.xyz',
        #'!overwrite' : True,  #VERY DANGEROUS, comment out when done using  
})

r2scan_3c_tddft = orca_tddft.copy() #let's only do one of these
r2scan_3c_tddft.update({
    **r2scan_3c,
    '!coords_from' : '../xtb_gfn2_opt_freq',
    '!xyz_file' : 'xtbopt.xyz',
})

crude_singlepoints = { #here we assign filenames to each group of settings
    'r2scan_3c_triplet_sp_orbitals' : r2scan_3c_triplet_sp,
    'r2scan_3c_bs_singlet_sp_orbitals_occs' : r2scan_3c_singlet_sp,
}

crude_tddft = {
    'r2scan_3c_tddft' : r2scan_3c_tddft,
}

gauss_nics = {} #does it even make sense to use Gaussian here?
#no, we'll validate w/ cam-b3lyp. We will wait until it is set up for ORCA
#for this
#we can run NICS in ORCA now, just need to know the keywords....
#also have to set up the TDDFT preprocessor here.


In [23]:
croot_dir ='/gpfs/research/alabuginlab/gage/michael/crude/'

# input_combi.do_everything(
#     [global_configs,xtb_cm_states,mike_mols,crude_confsearch_optfreq],
#     croot_dir,
# )

# input_combi.do_everything(
#     [global_configs,orca_cm_states,mike_mols,crude_singlepoints],
#     croot_dir,
# )


input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,crude_tddft],
    croot_dir,
)

Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nome2/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_tbucore/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_noby/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nomph/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_nome/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_npome/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_big_core/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_cby/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_ncmy/r2scan_3c_tddft
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/crude/_0_1_cm

In [24]:
cam_b3lyp_6_31gdp_opt_freq = orca_opt_freq.copy()
cam_b3lyp_6_31gdp_opt_freq.update({
        **cam_b3lyp,
        '!coords_from' : '../crest_gfn2_quick',
        '!xyz_file' : 'crest_best.xyz',
})

cam_b3lyp_6_31gdp_triplet_sp = orca_singlepoint.copy()
cam_b3lyp_6_31gdp_triplet_sp.update({
        **neutral_triplet,
        **cam_b3lyp,
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
        #'!overwrite' : True, #this is very hardcore, comment out unless using
})

cam_b3lyp_6_31gdp_singlet_sp = orca_singlepoint.copy()
cam_b3lyp_6_31gdp_singlet_sp.update({
        **neutral_singlet_brokensym,
        **cam_b3lyp,
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
        #'!overwrite' : True, #comment out until using, avoid accidentally removing files
})

cam_b3lyp_diffuse = cam_b3lyp.copy()
cam_b3lyp_diffuse.update({'basis' : '6-31+G(d,p)'})
cam_b3lyp_6_31_plus_gdp_tddft = orca_tddft.copy()
cam_b3lyp_6_31_plus_gdp_tddft.update({
    **cam_b3lyp_diffuse,
    'blocks' : diffuse_linear_dependency_blocks, #these all also failed for the same reason
    'scf_tolerance' : 'VeryTightSCF',
    '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
    '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
    '!overwrite' : 'all', #leaving this uncommented is like leaving your keys outside
})


########JOBS FOR DOING NICS
gauss_nics_prepro = gaussian_nics_preprocessing.copy()
gauss_nics_prepro.update({
    'xyz_file' :  'cam-b3lyp_6-31Gdp_opt_freq.xyz',
    '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
    '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
    #'!overwrite' : True, #these are fine, but we'll need to give some of them more time 
})

orca_nics_prepro = orca_nics_preprocessing.copy()
orca_nics_prepro.update({
    'xyz_file' :  'cam-b3lyp_6-31Gdp_opt_freq.xyz', #yes, we have to do this twice for these, once without '!' and once with
    '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
    '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
    #'!overwrite' : True,  DO NOT
})

cam_b3lyp_6_31_plus_gdp_NICS_gaussian = gaussian_nics.copy()
cam_b3lyp_6_31_plus_gdp_NICS_gaussian.update({
    **cam_b3lyp_diffuse,
    '!coords_from' : '../gaussian_prepro',
    '!xyz_file'    : 'gaussian_prepro.xyz', #I don't like the way this works. will change
    #'!overwrite' : True,  #DO NOT OVERWRITE
})
cam_b3lyp_6_31_plus_gdp_NICS_gaussian.update({
    'aux_basis' : None,
})

cam_b3lyp_6_31_plus_gdp_NICS = orca_nics.copy()
cam_b3lyp_6_31_plus_gdp_NICS.update({
    **cam_b3lyp_diffuse,
    'blocks' : diffuse_linear_dependency_blocks,
    'scf_tolerance' : 'VeryTightSCF',
    '!coords_from' : '../orca_prepro',
    '!xyz_file'    : 'orca_prepro.xyz',
    '!overwrite' : 'all', 
})

#SETTINGS FOR ITERATING IN INPUT_COMBI
crest_gfn2_quick = {
    'crest_gfn2_quick' : crest_quick_settings,
}

orca_cam_b3lyp_opt_freq_sp = {
    'cam-b3lyp_6-31Gdp_opt_freq' : cam_b3lyp_6_31gdp_opt_freq, 
    'cam-b3lyp_6-31Gdp_triplet_sp_orbitals' : cam_b3lyp_6_31gdp_triplet_sp,
    'cam-b3lyp_6-31Gdp_bs_sp_orbitals_occs' : cam_b3lyp_6_31gdp_singlet_sp,
}

orca_cam_b3lyp_tddft = {
    'cam-b3lyp_6-31+Gdp_tddft' : cam_b3lyp_6_31_plus_gdp_tddft,
}

gaussian_cam_b3lyp_nics = {
    'gaussian_prepro' : gauss_nics_prepro,
    'cam-b3lyp_6-31+Gdp_NICS_gaussian' : cam_b3lyp_6_31_plus_gdp_NICS_gaussian,
}

orca_cam_b3lyp_nics = {
    'orca_prepro' : orca_nics_prepro,
    'cam-b3lyp_6-31+Gdp_NICS' : cam_b3lyp_6_31_plus_gdp_NICS,
}


In [25]:
b3lyp_dir = '/gpfs/research/alabuginlab/gage/michael/quick/'

#b3lyp
input_combi.do_everything(
    [global_configs,xtb_cm_states,mike_mols,crest_gfn2_quick],
    b3lyp_dir,
)

input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,orca_cam_b3lyp_opt_freq_sp],
    b3lyp_dir,
)

input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,orca_cam_b3lyp_tddft],
    b3lyp_dir,
)

print(orca_nics)
input_combi.do_everything(
    [global_configs,orca_cm_states,mike_mols,orca_cam_b3lyp_nics],
    b3lyp_dir,
)
print(gaussian_nics)
input_combi.do_everything(
    [global_configs,gaussian_cm_states,mike_mols,gaussian_cam_b3lyp_nics],
    b3lyp_dir,
)


Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_nome2/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_tbucore/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_noby/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_nomph/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_nome/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_big_core/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_cby/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_ncmy/crest_gfn2_quick
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quic

/gpfs/home/gdb20/code/batch-manager/src/input_combi.py:131: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ledger.loc[identify_mask, 'job_id'] = f"{-1}"
/gpfs/home/gdb20/code/batch-manager/src/input_combi.py:131: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ledger.loc[identify_mask, 'job_id'] = f"{-1}"


overwriting directory: /gpfs/research/alabuginlab/gage/michael/quick/_0_1_nomph/cam-b3lyp_6-31+Gdp_NICS
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_nome/orca_prepro
JOB ALREADY SUCCEEDED
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/orca_prepro
overwriting directory: /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/cam-b3lyp_6-31+Gdp_NICS
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_big_core/orca_prepro
overwriting directory: /gpfs/research/alabuginlab/gage/michael/quick/_0_1_big_core/cam-b3lyp_6-31+Gdp_NICS
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_cby/orca_prepro
overwriting directory: /gpfs/research/alabuginlab/gage/michael/quick/_0_1_cby/cam-b3lyp_6-31+Gdp_NICS
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_ncmy/orca_prepro
overwriting directory: /gpfs/research/alabuginlab/gage/michael/quick/_0_

In [8]:
sym_failed_opt_keys = ['noby','npome','!directories']
time_failed_opt_keys = ['cmy','ome','oby','omy','!directories']
sym_failed_mols = {k : mike_mols[k] for k in sym_failed_opt_keys}
time_failed_mols = {k : mike_mols[k] for k in time_failed_opt_keys}

Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_noby/cam-b3lyp_6-31Gdp_opt_freq_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_noby/cam-b3lyp_6-31Gdp_triplet_sp_orbitals_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_noby/cam-b3lyp_6-31Gdp_bs_sp_orbitals_occs_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_noby/cam-b3lyp_6-31+Gdp_tddft_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/cam-b3lyp_6-31Gdp_opt_freq_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/cam-b3lyp_6-31Gdp_triplet_sp_orbitals_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/cam-b3lyp_6-31Gdp_bs_sp_orbitals_occs_cartesian
Job Directory NOT WRITTEN at /gpfs/research/alabuginlab/gage/michael/quick/_0_1_npome/cam-b3lyp_6-31+

In [11]:
failed_opt_configs = { '' : {
        'num_cores' : 12,
        'mem_per_cpu_GB' : 4,
        'runtime' : '4-00:00:00', #if this doesn't work I'll cry
        #'!max_jobs' : 10, #this doesn't do anything yet
        #'solvent' : 'thf', #this is what Mike told me we use
    }
}


In [12]:
input_combi.do_everything( #these all only requested 4 cores, that's a problem
    [failed_opt_configs,orca_cm_states,time_failed_mols,orca_b3lyp],
    b3lyp_dir,
)

NameError: name 'orca_b3lyp' is not defined

In [ ]:
crest = {
    'crest' : {
        'program' : 'CREST',
        'functional' : 'gfn2',
        'quick' : False,
        'cluster' : True,
        'noreftopo' : True,
    }
}

r2SCAN_3c_opt_freq = orca_opt_freq.copy()
r2SCAN_3c_opt_freq.update({
        **r2scan_3c,
        '!coords_from' : '../crest',
        '!xyz_file' : 'crest_best.xyz', 
})

wB97X_3c_sp_singlet = orca_singlepoint.copy()
wB97X_3c_sp_singlet.update({ #this one will include options for rendering orbitals,
    **neutral_singlet_brokensym,
    **wb97x_3c,
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : True, #hazardous to sanity to leave these uncommented when not using!
}),

wB97X_3c_sp_triplet = orca_singlepoint.copy()
wB97X_3c_sp_triplet.update({
    **neutral_triplet,
    **wb97x_3c,
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : True, #COMMENT THEM OUT UNTIL USING
})

wB97X_3c_tddft = orca_tddft.copy()
wB97X_3c_tddft.update({
    **wb97x_3c,
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : True,
})

orca_compound = {
    'r2SCAN-3c_opt_freq' : r2SCAN_3c_opt_freq,
    'wB97X-3c_sp_singlet' : wB97X_3c_sp_singlet,
    'wB97X-3c_sp_triplet' : wB97X_3c_sp_triplet,
    'wB97X-3c_tddft' : wB97X_3c_tddft,
}

In [13]:
orca_compound_dir = '/gpfs/research/alabuginlab/gage/michael/compound/'
#compound

input_combi.do_everything(
    [global_configs,thf,xtb_cm_states,mike_mols,crest],
    orca_compound_dir,
)
input_combi.do_everything(
    [global_configs,thf,orca_cm_states,mike_mols,orca_compound],
    orca_compound_dir,
)

NameError: name 'crest' is not defined

In [14]:
test_dir = '/gpfs/research/alabuginlab/gage/michael/test/'
test_molecules = {
    '0_1_omy' : {
        'xyz_file' : 'omy.xyz',
        'xyz_directory' : '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules',
    },
    '0_3_cby' : {
        'xyz_file' : 'cmy.xyz',
        'xyz_directory' : '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules',
    },
    '!directories' : True
}

cam_b3lyp_6_31gdp_opt_freq.update({
  '!overwrite' : True,  
})
orca_cam_b3lyp_opt_freq_sp = {
    'cam-b3lyp_6-31Gdp_opt_freq' : cam_b3lyp_6_31gdp_opt_freq, 
}
input_combi.do_everything( #these all only requested 4 cores, that's a problem
    [global_configs,test_molecules,orca_cam_b3lyp_opt_freq_sp],
    test_dir,
)
cam_b3lyp_6_31gdp_opt_freq.update({
    '!overwrite' : False,
})

     job_id                job_basename  \
218    -1.0  cam-b3lyp_6-31Gdp_opt_freq   

                                         job_directory   job_status program  \
218  /gpfs/research/alabuginlab/gage/michael/test/_...  not_started    ORCA   

             coords_from    xyz_filename  
218  ../crest_gfn2_quick  crest_best.xyz  
218    not_started
Name: job_status, dtype: object
<class 'pandas.core.series.Series'>
not_started
<class 'str'>
overwriting directory: /gpfs/research/alabuginlab/gage/michael/test/_0_1_omy/cam-b3lyp_6-31Gdp_opt_freq
    job_id                job_basename  \
242   -1.0  cam-b3lyp_6-31Gdp_opt_freq   

                                         job_directory   job_status program  \
242  /gpfs/research/alabuginlab/gage/michael/test/_...  not_started    ORCA   

             coords_from    xyz_filename  
242  ../crest_gfn2_quick  crest_best.xyz  
242    not_started
Name: job_status, dtype: object
<class 'pandas.core.series.Series'>
not_started
<class 'str'>
overwrit

/gpfs/home/gdb20/code/batch-manager/src/input_combi.py:136: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ledger.loc[identify_mask, 'job_id'] = f"{-1}"
